# Madina Tulegenova

In [10]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from functools import partial
from random import random
from sklearn.preprocessing import MinMaxScaler
from collections import defaultdict
from collections import Counter
from scipy.spatial import distance

## Iris dataset

In [11]:
df = load_iris()
df.data.shape

(150, 4)

In [12]:
def f_hash(w,r,b,x):
    return int((np.dot(w,x)+b)/r)

* https://docs.python.org/2/library/functools.html Here you can read about "partial"
* http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html About mapping to [0,1]

In [26]:
class KNNHash(object):
    def __init__(s,m,L,nn):
        s.m = m
        s.L = L
        s.nn = nn
    def fit(s,X,y):
        s.t_hh = [] #hash table
        for j in range(s.L):
            f_hh = [] #compositional hash function
            for i in range(s.m):
                w = np.random.rand(1,X[0].shape[0]) #  weights of a hash function
                f_hh.append(partial(f_hash,w = w,r=random(),b=random())) # list of initialized hash function
            s.t_hh.append(
                (defaultdict(list),f_hh)
            )
        for n in range(X.shape[0]): 
            for j in range(s.L):
                ind = 0
                for i in range(s.m):
                    ind = ind + s.t_hh[j][1][i](x=X[n]) #calculation of index in hash table, simply sum of all hash func
                s.t_hh[j][0][ind].append((X[n],y[n])) #saving sample into corresponding index    
    def predict(s,u):        
        distClosest = 0
        distArray = []        
        for j in range(s.L):
            inds = []
            for i in range(s.m):
                inds.append(s.t_hh[j][1][i](x=u))
            cntr = Counter([outp for inpt,outp in s.t_hh[j][0][sum(inds)]])
            print(cntr)
            
            #my peace of code
            for k in range(len(s.t_hh[j][0][sum(inds)])):
                distArray.append(distance.euclidean(u,s.t_hh[j][0][sum(inds)][k][0])) #finding the euclidean distance and adding it to new array
        distClosest = max(distArray) #identify the maximum distance in the array
        print("closest dist: ", distClosest)

In [27]:
scaler = MinMaxScaler()
scaler.fit(df.data)
x = scaler.transform(df.data)
y = df.target

In [28]:
knnhash = KNNHash(4,4,4)
testing1x = x[0]
testing2x = x[75]
testing3x = x[149]

testing1y = y[0]
testing2y = y[75]
testing3y = y[149]
x = np.delete(x,[0,75,149],axis=0)
y = np.delete(y,[0,75,149],axis=0)

knnhash.fit(x,y)
print(testing1y)
knnhash.predict(testing1x)
print("-------------")
print(testing2y)
knnhash.predict(testing2x)
print("-------------")
print(testing3y)
knnhash.predict(testing3x)

0
Counter({0: 13})
Counter({0: 31, 1: 4})
Counter({0: 23})
Counter({0: 38, 1: 1})
closest dist:  0.7932184122039291
-------------
1
Counter({1: 9, 2: 2})
Counter({1: 12, 2: 3})
Counter({1: 11, 2: 9})
Counter({2: 33, 1: 10})
closest dist:  0.553336221927024
-------------
2
Counter({1: 5, 2: 3})
Counter({2: 13, 1: 6})
Counter({1: 8, 2: 2})
Counter({1: 29, 2: 9, 0: 1})
closest dist:  1.0275936860924342


* Each string above corresponds to the particular hash table. And index in counter maps to the class. For example Counter({0: 13, 1: 1}) means that there are 13 samples close to "u" with "0" class labels and 1 sample with "1" class label.